In [1]:
import argparse
import os
import sys

from astropy.io import fits

import pint.models as models
import pint.fermi_toas as fermi
import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import SkyCoord

import scipy.optimize as op
import scipy.interpolate as interpolate
import scipy.stats as ss
import matplotlib.pyplot as plt

import pint.fitter as fitter
import pint.toa as toa
# import presto.psr_utils as pu
import pint.derived_quantities as pd
import pint.residuals as pr
import pint.simulation as ps
from pint.utils import FTest
from pint.plot_utils import phaseogram
from pint.eventstats import hmw
from pint.models import parameter as pp

from scipy.linalg import cho_solve,cho_factor,solve_triangular,LinAlgError

import astropy.time
from astropy.time import TimeDelta
from astropy.visualization import quantity_support

import copy
from pint.fitter import MaxiterReached, StepProblem
from pint.utils import FTest

import pint.logging
from loguru import logger as log

pint.logging.setup(level='INFO')

1

In [2]:
def designmatrix(mjds,nwaves,setpoint,F0,extended=0):
    ndata = len(mjds)
    Tobs = (mjds[-1]-mjds[0]+extended)/365.25
    tnscale = (2/Tobs)**0.5 * F0
    freqs = np.linspace(1/Tobs,nwaves/Tobs,nwaves)
    F = np.empty([ndata,2*nwaves],dtype=np.float64)
    phase= np.empty_like(mjds)
    times = (mjds - setpoint)/365.25 
    #times = (mjds + extended - setpoint)/365.25 ? do I need to extend dt as well?
    for iharm, freq in enumerate(freqs):
        phase[:] = (2*np.pi*freq)*times
        np.cos(phase,out=F[:,2*iharm])
        np.sin(phase,out=F[:,2*iharm+1])
    F *= tnscale
    return freqs, F, Tobs, times, tnscale

def joint_designmatrix(model,ts,nwaves,extended=0):
    mjds = ts.get_mjds().value
    
    freqs, F, Tobs, times, tnscale = designmatrix(mjds,nwaves,model.PEPOCH.value,model.F0.value,extended)
    
    D, params, units = model.designmatrix(ts)
    D *= -model.F0.value
    D = D.transpose()

    output = np.empty((D.shape[0]+F.T.shape[0],D.shape[1]),dtype=np.float64)
    output[:D.shape[0]]=D
    output[D.shape[0]:]=F.T
    return output, F, freqs

In [2]:
class TNMatrix(object):
    """ Encapsulate the spectral decomposition form of a TN process."""
    def __init__(self,p,freqs,model,ts,zeropad=None):
        """ NB freqs should be in cycles/year!."""
        self.freqs = freqs
        n = zeropad + 2*len(freqs)
        self._H = np.zeros((n,n),dtype=float)
        x,y = np.diag_indices_from(self._H)
        self._x = x[zeropad:]
        self._y = y[zeropad:]
        self.update_p(p)

        self.model = model
#         self.epoch = model.WAVEEPOCH.value if model.WAVEEPOCH else model.PEPOCH.value
        self.epoch = model.PEPOCH.value
        self.mjds = ts.get_mjds().value
        self.scale = (2/(self.mjds[-1]-self.mjds[0])/365.25)**0.5*self.model.F0.value

    def designmatrix(self,mjds):
        ndata = len(mjds)
        nharm = len(self.freqs)
        F = np.empty([2*nharm,ndata],dtype=np.float128)
        phase = np.empty_like(mjds)
        times = mjds-self.epoch
        for iharm,freq in enumerate(self.freqs):
            phase[:] = (2*np.pi*freq)*times
            np.cos(phase,out=F[2*iharm,:])
            np.sin(phase,out=F[2*iharm+1,:])
        F *= self.scale
        return F
    
    def eval_pl(self,p,freqs):
        amp,alpha = p
        return (10**amp)*freqs**-alpha
    
    def get_values(self):
        return self.coeffs
    
    def H(self):
        """ Return inverse covariance matrix."""
        return self._H
    
    def update_p(self,p):
        tn_vals = 2./eval_pl(p,self.freqs)
        self._H[self._x[::2],self._y[::2]] = tn_vals
        self._H[self._x[1::2],self._y[1::2]] = tn_vals
        self._p = p

In [ ]:
class GaussianFit(object):
    def __init__(self,model,ts,coeffs,nwaves,zeropad=None):
        self.model = model
        self.ts = ts
        self.coeffs = coeffs
        self.nwaves = nwaves
        self.pulsen = self.ts.get_pulse_numbers()
        self.errs = self.ts.get_errors() # Should this be changed to weights if we are dealing with photons?
        self.h_vec = 1./self.errs**2
        
    def get_joint_phase(self,coeffs,extended=0):
        mjds = self.ts.get_mjds().value
        freqs, F, Tobs, times, tnscale = designmatrix(mjds,nwaves,self.model.PEPOCH.value,extended=extended)
        tn_phase = np.inner(F,coeffs)
        tm_phase = self.model.phase(self.ts)
        tm_phase = (tm_phase[0]+tm_phase[1]).value
        return tn_phase+tm_phase
    
    def data_matrices(self,extended=0):
        M,F,freqs=joint_designmatrix(self.model,self.ts,self.nwaves,extended=extended)
        phase = self.get_joint_phase(extended=extended)
        est_resid = self.pulsen - phase
        
        # G is the joint designmatrix * the errors * residuals, does this make sense with weights?
        G = np.inner(self.M,est_resid*self.h_vec)
        
        # Hlambda matrix?
        Hlambda = np.asarray((self.M*self.h_vec)@M.T)
        
        return phase, est_resid, M, G, H
    
    def get_values(self,offset,coeffs):
        fitvals = np.append(offset,[getattr(model,p).value for p in model.free_params])
        values = np.append(coeffs)
        return values
    
    def lnlikelihood(self,offset,coeffs,extended=0):
        values = self.get_values(offset,coeffs)
        